# snowmobile

```{include} /description.md
```

```{eval-rst}

.. toctree::
    :maxdepth: 1
    :hidden:

    ./setup.md

.. toctree::
    :caption: Usage
    :maxdepth: 1
    :hidden:
    
    ./usage/connector.md
    ./usage/table.md
    ./usage/script.md
    ./usage/sql.md
    ./usage/snowmobile_toml.md

.. toctree::
    :caption: Additional Resources
    :maxdepth: 1
    :hidden:

    ./modindex.md
    ./extensions.md
    ./usage/advanced.md
    ./snippets.md

.. toctree::
    :caption: Meta
    :maxdepth: 1
    :hidden:

    ./acknowledgements.md
    ./changelog.md
    ./authors.md
    ./license.md
```

---

---

<br>

<p class="hanging"><h2>Example</h2></p>

- *[Connecting](#connecting)*
- *[Loading Data](#loading-data)*
- *[Executing Queries](#executing-queries)*
- *[Working with SQL Scripts](#working-with-sql-scripts)*
- *[Information API](#information-api)*

<br>

### *Connecting*

````{admonition} snowmobile.connect()
:class: toggle, note, is_explanation
{func}`snowmobile.connect()` is an alias for {class}`snowmobile.Connector()`; its purpose is provide an entry point that will:
1.  Locate and instantiate [snowmobile.toml](./snowmobile_toml.md#snowmobiletoml) 
    as a {class}`~snowmobile.Configuration` object ({class}`sn.cfg`)
1.  Establish a connection to {xref}`snowflake`
1.  Store the {xref}`SnowflakeConnection` ({class}`sn.con`) and execute commands against the database 

+++

The first time it's invoked, {xref}`snowmobile` will find [*snowmobile.toml*](./usage/snowmobile_toml) and cache its location; 
this step isn't repeated unless the file is moved, the cache is manually cleared, or a new version of {xref}`snowmobile` is installed.

+++

**With all arguments omitted, it will authenticate with the defaults specified in** [*snowmobile.toml*](./usage/snowmobile_toml) **and return a live** [**Connector**](./usage/connector.md).

+++

<hr class="sn-green">

{link-badge}`./usage/connector.html,cls=badge-primary badge text-white,Usage: snowmobile.Connector,tooltip=Intro & usage documentation for snowmobile.Connector`
{link-badge}`./autoapi/snowmobile/core/connector/index.html,cls=badge-secondary badge-pill text-white,API Docs: snowmobile.core.connector,tooltip=Documentation parsed from docstrings`
````

In [1]:
import snowmobile

sn = snowmobile.connect()

Locating credentials...
(1 of 2) Checking for cached path...
(2 of 2) Cached path found at ../Snowmobile/snowmobile.toml
..connected: snowmobile.Connector(creds='default')


In [ ]:
# Or with a different configuration
sn_sandbox = snowmobile.connect(creds='sandbox')  

---

---

<br>

### *Loading Data*

`````{admonition} snowmobile.Table
:class: toggle, is_explanation

{class}`~snowmobile.Table` is a data loading solution that at minimum accepts a
[**Connector**](./usage/connector) ({class}`sn`), a {class}`~pandas.DataFrame` ({class}`df`),
and a table name ({class}`table`: {class}`str`).

````{tabbed} Context
In the same way that [**Connector**](./usage/connector) handles its keyword arguments,
{class}`~snowmobile.Table` will adhere to any arguments explicitly provided and defer
to the values configured in [*snowmobile.toml*](./usage/snowmobile_toml) otherwise.

+++

`as_is=True` tells {xref}`snowmobile` to invoke {meth}`~snowmobile.Table.load()` with
the arguments provided as opposed to inspecting `t1` for information on the existence
of the table or its compatability with the {class}`df` we're loading.

+++

The behavior outlined below reflects those within the
[default **snowmobile.toml** file](./usage/snowmobile_toml.md#file-contents).
````

````{tabbed} df
The `df` used in this example can be created with:
```python
"""Generate dummy DataFrame for snowmobile.Table example."""
import pandas as pd
import numpy as np

df = pd.DataFrame(
    data = {'col1': np.ones(3), 'col2': np.zeros(3)}
)

# -- complete snippet; should run 'as is' --
```
<hr class="sn-spacer">
````

````{tabbed} +
**In the below example,** {class}`~snowmobile.Table` **will:**
1. Check if *sample_table* exists in the schema associated with {attr}`sn.con`
2. If *sample_table* already exists, it will validate the columns of {class}`df` compared to those in *sample_table*
    - How it handles the outcome of this validation is determined by the value of `if_exists` in the *[loading.default-table-kwargs]*
        section of [*snowmobile.toml*](./usage/snowmobile_toml)
3. If *sample_table* does not exist, it will generate DDL from {class}`df` and execute it as part of the loading process
````

<hr class="sn-green">

{link-badge}`./usage/table.html,cls=badge-primary badge text-white,Usage: snowmobile.Table,tooltip=Intro & usage documentation for snowmobile.Table`
{link-badge}`./autoapi/snowmobile/core/table/index.html,cls=badge-secondary badge-pill text-white,API Docs: snowmobile.core.table,tooltip=Documentation parsed from docstrings`
`````

In [2]:
# Verify table does not exist before moving forward with example
if sn.sql.exists('sample_table'):
    sn.sql.drop('sample_table')

In [10]:
"""Generate dummy DataFrame for snowmobile.Table example."""
import pandas as pd
import numpy as np

df = pd.DataFrame(
    data = {'col1': np.ones(3), 'col2': np.zeros(3)}
)
df.head()

,col1,col2
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0


In [11]:
t1 = snowmobile.Table(sn=sn, df=df, table='sample_table', as_is=True)

Loading into 'gem7318.SAMPLE_TABLE`..
(1 of 4)
	CREATE OR REPLACE TABLE SAMPLE_TABLE ( ..
(2 of 4)
	create stage SAMPLE_TABLE_stage file_format = snowmobile_default_psv;
(3 of 4)
	put file://C:/Users/GEM7318/Documents/Github/Snowmobile/docs/sample_table.csv @SAMPLE_TABLE_stage
	auto_compress = true
(4 of 4)
	copy into SAMPLE_TABLE from @SAMPLE_TABLE_stage
	on_error = continue
..completed: 3 rows in 2 seconds


---

<br>

---

### *Executing Queries*

`````{admonition} sn: snowmobile.Connector
:class: toggle, is_explanation

{xref}`snowmobile` provides three convenience methods for executing raw SQL directly off the {class}`~snowmobile.Connector`.

<hr class="sn-green">

{meth}`~snowmobile.Connector.query()` implements {meth}`pandas.read_sql()` for querying results into a {class}`~pandas.DataFrame`\
{link-badge}`./autoapi/snowmobile/core/connector/index.html#snowmobile.core.connector.Connector.query,cls=badge-secondary badge-pill text-white,API Docs: Connector.query(),tooltip=Documentation parsed from module docstring`

+++

{meth}`~snowmobile.Connector.ex()` implements {meth}`SnowflakeConnection.cursor().execute()` for executing commands within a {xref}`SnowflakeCursor`\
{link-badge}`./autoapi/snowmobile/core/connector/index.html#snowmobile.core.connector.Connector.ex,cls=badge-secondary badge-pill text-white,API Docs: Connector.ex() ,tooltip=Documentation parsed from module docstring`

+++

{meth}`~snowmobile.Connector.exd()` implements {meth}`SnowflakeConnection.cursor(DictCursor).execute()` for executing commands within a {xref}`DictCursor`\
{link-badge}`./autoapi/snowmobile/core/connector/index.html#snowmobile.core.connector.Connector.exd,cls=badge-secondary badge-pill text-white,API Docs: Connector.exd(),tooltip=Documentation parsed from module docstring`

+++

<hr class="sn-green">

{link-badge}`./usage/connector.html#executing-raw-sql,cls=badge-primary badge text-white,Usage: Executing Raw SQL,tooltip=Usage documentation for Executing Raw SQL`
{link-badge}`./autoapi/snowmobile/core/connector/index.html,cls=badge-secondary badge-pill text-white,API Docs: snowmobile.core.connector,tooltip=Documentation parsed from docstrings`
`````

In [5]:
# Into a DataFrame
sn.query('select * from sample_table')

,col1,col2
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0


In [16]:
# Into a SnowflakeCursor
sn.ex('select * from sample_table').fetchall()

[(1.0, 0.0), (1.0, 0.0), (1.0, 0.0)]

In [20]:
# Into a DictCursor
sn.exd('select * from sample_table').fetchall()

[{'COL1': 1.0, 'COL2': 0.0},
 {'COL1': 1.0, 'COL2': 0.0},
 {'COL1': 1.0, 'COL2': 0.0}]

---

<br>

---

### *Working with SQL Scripts*

`````{admonition} snowmobile.Script
:class: toggle, is_explanation

[**snowmobile.Script**](./usage/script.ipynb) imports a sql file and parses its contents according to its structure.

````{tabbed} Context
**At a minimum, the file is split into individual statements which are checked for valid tags;
if a tag is not found, [Script](./usage/script.ipynb) will generate one based on
the literal first SQL keyword it contains and its index position (e.g. `select~data-statement #3` in *sample_table.sql*)**.

+++

By providing {class}`script` (below) the same instance of {class}`sn` with which {class}`t1` (above)
was instantiated, **the {xref}`SnowflakeConnection` and [Configuration](./usage/snowmobile_toml) is
shared amongst:**
- {class}`sn:` {class}`~snowmobile.Connector`
- {class}`t1:` {class}`~snowmobile.Table`
- {class}`script:` {class}`~snowmobile.Script`

````

````{tabbed} sample_table.sql
This example makes use of the following `path` to **sample_table.sql**:
```python
"""Store path to 'sample_table.sql' in `path` variable."""
from pathlib import Path

path = Path.cwd() / 'snippets' / 'getting_started' / 'sample_table.sql'
```
<br>

The contents of **sample_table.sql** are:
```{eval-rst}
.. literalinclude:: ./snippets/getting_started/sample_table.sql
   :language: sql
   :lines: 1-17
```
<hr class="sn-spacer">
````

<hr class="sn-green">

{link-badge}`./usage/script,cls=badge-primary badge text-white,Usage: snowmobile.Script,tooltip=Intro & usage documentation for snowmobile.Script`
{link-badge}`./autoapi/snowmobile/core/script/index.html,cls=badge-secondary badge-pill text-white,API Docs: snowmobile.core.script,tooltip=Documentation parsed from docstrings`
`````

In [6]:
# Setup
from pathlib import Path

path = Path.cwd() / 'snippets' / 'snowmobile' / 'sample_table.sql'

<p style="margin-left: 0.9rem; margin-top: 1em; font-size: 0.75rem; font-weight: 800;">Setup</p>

```{div} sn-indent-cell, sn-hanging 
Setup
```

In [12]:
script = snowmobile.Script(sn=sn, path=path)
print(script)

snowmobile.Script('sample_table.sql')


In [13]:
# High-level view of parsed contents
script.dtl()  

sample_table.sql
1: Statement('create transient table~any_random_table1')
2: Statement('create table~sample_table')
3: Statement('select data~statement #3')
4: Statement('select data~sample_table')


<p class="sn-indent-cell">Accessing Statements</p>

In [14]:
# Individual statements can be accessed by index position
script(4)

Statement('select data~sample_table')

In [15]:
# ..including negative indexing
script(-1)  

Statement('select data~sample_table')

In [16]:
# ..or by their tag as a string
script('select data~sample_table')  

Statement('select data~sample_table')

<p class="sn-indent-cell">Executing Statements</p>

In [41]:
# They can be run directly off the Script
script.run(4, render=True)
script(4).results.head(1)

```sql
select * from sample_table;
```

,col1,col2
0,1.0,0.0


In [27]:
# ..or stored and manipulated on their own
s4 = script(4).run(render=True)
s4.results.head(1)

```sql
select * from sample_table;
```

,col1,col2
0,1.0,0.0


&nbsp;&nbsp;See [Advanced Examples](./usage/advanced.md) for more in-depth applications of {class}`~snowmobile.Script`.

In [40]:
with script.filter(incl_kw=['create'], excl_nm=['create table']) as s:
    s.dtl()

print('')

script.dtl()

sample_table.sql
1: Statement('create transient table~any_random_table1')

sample_table.sql
1: Statement('create transient table~any_random_table1')
2: Statement('create table~sample_table')
3: Statement('select data~statement #3')
4: Statement('select data~sample_table')


In [53]:
script(2).run(render=True)

```sql
create or replace table sample_table (
	col1 int,
	col2 int
);
```

Statement('create table~sample_table')

In [34]:
print(script.s(2).sql)

create or replace table sample_table (
	col1 int,
	col2 int
)


In [35]:
print(script.s('create table~sample_table').sql)

create or replace table sample_table (
	col1 int,
	col2 int
)


In [23]:
with script.filter(incl_nm=['.*sample_table']) as s:
    s.dtl()

sample_table.sql
1: Statement('create table~sample_table')


<br>

---

---

### *Information API*

`````{admonition} snowmobile.SQL
:class: toggle, is_explanation

{class}`snowmobile.SQL` generates and executes raw SQL from inputs; 
it comes free as the {attr}`~snowmobile.Connector.sql` attribute of 
{class}`~snowmobile.Connector`, and its purpose is to provide a simple
Python API to query metadata and execute administrative commands 
against {xref}`snowflake`.

<hr class="sn-green">

{link-badge}`./usage/sql,cls=badge-primary badge text-white,Usage Docs: snowmobile.SQL,tooltip=Intro & usage documentation for snowmobile.SQL`
{link-badge}`./autoapi/snowmobile/core/sql/index.html,cls=badge-secondary badge-pill text-white,API Docs: snowmobile.core.sql,tooltip=Documentation parsed from docstrings`
`````

<p class="sn-indent-cell"></p>

In [5]:
# Check existence of tables/views
sn.sql.exists('sample_table')

True

In [2]:
# Sample 'n' records from a table
sn.sql.table_sample('sample_table', n=-1)

,col1,col2
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0


In [3]:
# Submit basic administrative commands
sn.sql.clone(nm='sample_table', to='sample_table2')

,status
0,Table SAMPLE_TABLE2 successfully created.


In [4]:
# Check depth
sn.sql.cnt_records('sample_table2')

3

In [5]:
# Fetch columns
sn.sql.columns('sample_table2')

['COL1', 'COL2']

In [53]:
# Query DDL
print(sn.sql.ddl('sample_table'))

create or replace TABLE SAMPLE_TABLE (
	COL1 FLOAT,
	COL2 FLOAT
);


In [7]:
# Provide `run=False` to get the raw sql
drop_sql = sn.sql.drop('sample_table', run=False)

print(type(drop_sql))
print(drop_sql)

<class 'str'>
drop table if exists GEM7318.SAMPLE_TABLE


<p class="sn-indent-cell"></p>

<p class="sn-indent-cell">Cleaning up after ourselves can be done with:</p>

In [8]:
for t in ['sample_table', 'sample_table2']:
    sn.sql.drop(t)

<p class="sn-indent-cell"></p>

---

# <u>Meta / Non-Output</u>
---

All cells below this are either excluded from output via the `remove-cell` cell-tag
or contain contents that will not visibly render in the output.

# Style
---

Reduce vertical spacing between h2-h3 headings for *Examples*:
```css
<style>
.md-typeset h2, .md-typeset h3 {
    margin-top: -0.5rem;
}
</style>
```

<style>
.md-typeset h2, .md-typeset h3 {
    margin-top: -0.5rem;
}
</style>